# CREANDO ACCESO A DRIVE PARA LA PREPARACION LOS DATOS

In [0]:
#CREACION DE CARPETA CON LA INFORMACION DEL DRIVE QUE INICIE SESION
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#MANEJO DE CARPETAS
%cd /
%cd content/drive/My Drive/Todo_Tesis/LUCAS 

!pip install pydrive
!git clone https://github.com/thtrieu/darkflow.git
!pip install Cython 
!pip install pytube
!pip install PyDrive
!pip install python-utils

#CREACION DE CARPETA EN CASO QUE NO EXISTA Y UBICARSE EN ESA
import os
os.chdir('/content/drive/My Drive/Todo_Tesis/LUCAS/darkflow') ## Cambiar
!pwd

!pip install -e .
# DESCARGAR CODIGO QUE SE ENCUENTRAN EN GITHUB
!git clone https://github.com/StraigenDaigen/Daigenyolo.git

!pwd

!pip install -e .

In [0]:
#IMPORTAR LIBRERIAS
from darkflow.net.build import TFNet
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
import pydrive
from tqdm import tqdm


In [0]:
import pydrive
import time
import os
from tqdm import tqdm
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

#AUTENTIFICION DE USUARIS QUE UTILIZARA LOS SERVICOS DE LA NUBE
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# CARGAR LOS PESOS DE ENTRENAMIENTO PARA EL YOLO

newpath = '/content/drive/My Drive/Todo_Tesis/LUCAS/darkflow/bin'
if not os.path.exists(newpath):
  os.makedirs(newpath)
  
#DESCARGAR LOS PESOS QUE SE ENCUENTRAN EN UNA CARPETA DE DRIVE

download = drive.CreateFile({'id': '1rSne0luPzETpuSHk742ezr4aX0FCWcBV'}) # insert the fild ID here
download.GetContentFile('/content/drive/My Drive/Todo_Tesis/LUCAS/darkflow/bin/yolov2.weights')

In [0]:
#OPCIONES DEL YOLO
options = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolov2.weights',
    'threshold': 0.1,
    #'demo': '/content/sample_data/cafeteriamedia',
    'gpu' : 0.8
          }

In [0]:
#INICIALIZAR OPCIONES 
%cd /content/drive/My Drive/Todo_Tesis/LUCAS/darkflow
tfnet = TFNet(options)

# METODOS  ESTANDARIZADOS DE YOLO

In [0]:
def predict_img():
  
  #result = tfnet.return_predict(link)
  result = tfnet.return_predict()
   

  for n,value in enumerate(result):
    
    tl = (result[n]['topleft']['x'], result[n]['topleft']['y'])
    br = (result[n]['bottomright']['x'], result[n]['bottomright']['y'])
    label = result[n]['label']
    img = cv2.rectangle(img,tl,br,(0,255,0))
    
  plt.figure(figsize=(20,10))    
  plt.imshow(img)

In [0]:
def create_image_paths():
  
  import glob
  # Get all image file paths to a list.
  images = list(glob.iglob(os.path.join('/content/drive/My Drive/Todo_Tesis/LUCAS/sample_data_frames/',title, '*.*')))
  # Sort the images by name index.
  images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))
  
  return images

In [0]:
def fps():  
  
  video = cv2.VideoCapture(os.path.join('/content/drive/My Drive/Todo_Tesis/LUCAS/sample_data/',title))  
  fps = video.get(cv2.CAP_PROP_FPS)
  print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
  video.release()
  return fps

In [0]:
def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
    """
    from tqdm import tqdm
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in tqdm(images):
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

#SACAR LOS FRAMES DE LOS VIDEOS

In [0]:
def sample_frames():
  import os
  capture = cv2.VideoCapture(os.path.join('/content/drive/My Drive/Todo_Tesis/LUCAS/sample_data/',title)) 

  
  frames = []
  frame_count = 0

  video_frame_path = os.path.join('/content/drive/My Drive/Todo_Tesis/LUCAS/sample_data_frames/',title)
  
  if not os.path.exists(video_frame_path):
    os.makedirs(video_frame_path)
    print('Frames Directory Created...')
  
  while True:
  
    ret, frame = capture.read()
    frames.append(frame)
    frame_count += 1
  
  
    if not ret:
      print(frame_count)
      return (frames,frame_count,video_frame_path)
      break

In [0]:
def predict_video():
  
  global title
  
  #title = download_youtube_video()
  title = video_subido()
   
  frames,frame_count,video_frame_path = sample_frames()
  
  #load video and get every frame of it for analysis..
  

  
  # for every frame run yolo prediction and save the frame as jpg file

  for count,frame in tqdm(enumerate(frames[1:-10])):
    

    #frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    
    results = tfnet.return_predict(frame)
  
    for n,value in enumerate(results):
      
      tl = (results[n]['topleft']['x'], results[n]['topleft']['y'])
      br = (results[n]['bottomright']['x'], results[n]['bottomright']['y'])
      label = results[n]['label']
      if label != 'person': 
        continue
      label = 'Persona' 
      
      sum= tl[0] + tl[1]
      sum2= br[0] + br[1]
      #sum = br[0] 
      
      if sum > 800 or sum2>800:      #filtro para minimizar rectangulos grandes
     # if sum > 400: 
        continue      
#cv2.rectangle(img,(384,0),(510,128),(0,255,0),3)
      frame = cv2.rectangle(frame,tl,br,(255,255,255),1)
   #   frame = cv2.resize(frame,(1280,500))
      
      #frame = cv2.putText(frame,label,tl,cv2.FONT_HERSHEY_SIMPLEX,0.4,(255,255,255),1)    #BGR  BLUE GREEN RED
      
      
      #frame = cv2.addText(label)
      #frame = cv2.line(img,(0,frame.shape[0]//2),(600,img.shape[0]//2),(255,0,0),2)   
      
  
    name = '{0}.jpg'.format(count)
    
    name = os.path.join(video_frame_path,name)  
    cv2.imwrite(name, frame)
    
  print('All Frames Saved...')
 # print ("tl:", tl, "br", br) 
  
  newpath = '/content/drive/My Drive/Todo_Tesis/LUCAS/sample_data_out/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)
    
  images = create_image_paths()
  #print(images)
  frames_per_second = fps()
  make_video(os.path.join('/content/drive/My Drive/Todo_Tesis/LUCAS/sample_data_out/',title), images, fps=frames_per_second)

#CODIGO DE INGRESO A LOS VIDEOS UNO POR UNO DEPENDIENDO EL NOMBRE

In [0]:
def video_subido():
  global cll,cm,cv,ell,em,ev,aull,aum,auv

  
 # streams=uploaded.default_filename

  #title = 'cafeto_vacia.mp4'     #se debe cambiar cada vez que se suba un archivo nuevo
  
  import os
  path ='/content/drive/My Drive/Todo_Tesis/LUCAS/sample_data'
  for file in os.listdir(path):
    #CAFETERIA
    if file.endswith("cafeto_llena.mp4"):
      if cll=='1':
        print("encontro"+ file+"entro")
        title = file
        cll='0'
        return title
        break
      
    if file.endswith("cafeto_media.mp4"):
      if cm=='1':
        print("encontro"+ file+"entro")
        title = file
        cm='0'
        return title
        break
      
    if file.endswith("cafeto_vacia.mp4"):
      if cv=='1':
        print("encontro"+ file+"entro")
        title = file
        cv='0'
        return title
        break
        
    #ELECTRÓNICA   
        
    if file.endswith("electro_llena.mp4"):
      if ell=='1':
        print("encontro"+ file+"entro")
        title = file
        ell='0'
        return title
        break        
      
    if file.endswith("electro_media.mp4"):
      if em=='1':
        print("encontro"+ file+"entro")
        title = file
        em='0'
        return title
        break
      
    if file.endswith("electro_vacia.mp4"):
      if ev=='1':
        print("encontro"+ file+"entro")
        title = file
        ev='0'
        return title
        break
  
        
   #AUTOMÁTICA     
      
    if file.endswith("automatica_vacia.mp4"):
      if auv=='1':
        print("encontro"+ file+" : entro")
        title = file
        auv='0'
        return title
        break
    if file.endswith("automatica_llena.mp4"):
      if aull=='1':
        print("encontro"+ file+"entro")
        title = file
        aull='0'
        return title
        break
      
    if file.endswith("automatica_media.mp4"):
      if aum=='1':
        print("encontro"+ file+"entro")
        title = file
        aum='0'
        return title
        break
  print("No hay archivos con los nombres solicitados")
  title = ' '     #se debe cambiar cada vez que se suba un archivo nuevo
  return title

# METODO DE OPCIONES DE ELECCION MANUAL DE VIDEOS FALTANTES DONDE (0)SIGNIFICA QUE YA ESTA Y (1) QUE HACE FALTA

In [0]:
global cll,cm,cv,ell,em,ev,aull,aum,auv
#Cafeteria
cll='0'
cm='0'
cv='0'
#electronica
ell='0'
em='0'
ev='0'
#Automatica
aull='0'
aum='0'
auv='0'

# INICIO DE TODOS LOS METODOS DE YOLO EL CULA SACARA LOS FRAMES DE LOS VIDOS SEGUN SU NOMBRE Y POSTERIORMENTE IDENTIFICARA LAS PERSONAS EN CADA FRAME

In [0]:
for a in range(0 , 8):
  predict_video()

# METODOS TFP

In [0]:
!cp -r /content/drive/My Drive/Todo_Tesis_Final/Dataset/Dataset_listo/ /content/drive/My Drive/Todo_Tesis_Final/tensorflow-for-poets-2/tf_files/ocupacion


#ENTRENAMIENTO DE LA RED NEURONAL DE CLASIFICACION

%cd /content/drive/My Drive/Todo_Tesis_Final/tensorflow-for-poets-2
!python -m scripts.retrain \
  --bottleneck_dir=tf_files/bottlenecks \
  --how_many_training_steps=5000 \ # variar al gusto
  --model_dir=tf_files/models/ \
  --summaries_dir=tf_files/training_summaries/"mobilenet_0.50_224" \
  --output_graph retrained_graph.pb\
  --output_labels retrained_labels.txt\
  --architecture "mobilenet_0.50_224" \
  --image_dir=tf_files/

#DIRECCIONAMOS LOS GRPHS Y LABELS A LA CARPETA QUE DEBEN ESTAR LA CUAL ES TF_FILES
!mv retrained_graph.pb tf_files
!mv retrained_labels.txt tf_files

Busqueda de documentos

In [0]:
%cd /content/drive/My Drive
path='/content/drive/My Drive'
import os
print("inicio")
for file in os.listdir(path):
  print("bus")
  if file.endswith("Todo_Tesis_Final"):
    print("encontre")
 